In [1]:
import numpy as np
import pandas as pd

# Normalize 2010 - 2020 data

In [2]:
pd.read_csv('../data/batting_2010_2020.csv').columns

Index(['last_name', ' first_name', 'year', 'b_ab', 'b_total_hits', 'b_double',
       'b_triple', 'b_home_run', 'b_strikeout', 'b_walk', 'b_rbi',
       'r_total_stolen_base', 'b_game', 'b_hit_by_pitch', 'r_run',
       'b_total_sacrifices', 'Unnamed: 16'],
      dtype='object')

In [3]:
batting_df = pd.read_csv('../data/batting_2010_2020.csv')
display(batting_df.columns)
batting_df['player_id'] = (batting_df[' first_name'] + batting_df['last_name']).str[1:]
batting_df = batting_df[['player_id', 'b_ab', 'b_walk', 'b_double', 'b_game', 'b_total_hits', 'b_hit_by_pitch', 'b_home_run', 'r_run', 'b_rbi', 'r_total_stolen_base', 'b_total_sacrifices', 'b_strikeout', 'b_triple', 'year']]
batting_df = batting_df[batting_df['year'] != 2020]

Index(['last_name', ' first_name', 'year', 'b_ab', 'b_total_hits', 'b_double',
       'b_triple', 'b_home_run', 'b_strikeout', 'b_walk', 'b_rbi',
       'r_total_stolen_base', 'b_game', 'b_hit_by_pitch', 'r_run',
       'b_total_sacrifices', 'Unnamed: 16'],
      dtype='object')

In [4]:
yearly_means = batting_df.groupby('year').mean()
yearly_stds = batting_df.groupby('year').std()

In [5]:
batting_df_norm_year = batting_df.apply(lambda row: (row - yearly_means.loc[row['year']]) / yearly_stds.loc[row['year']], axis=1)
batting_df_norm_year['year'] = batting_df['year']
batting_df_norm_year['player_id'] = batting_df['player_id']

In [6]:
batting_df_norm_year.to_csv('../data_normalized/batting_norm_2010_2020.csv')

# Group into agg / ts for interesting players

In [23]:
ids_to_test = ('GeorgeSpringer', 'MikeTrout', 'BrettGardner', 'JoeyVotto', 'AaronJudge', 'FernandoTatis Jr.', 'MookieBetts', 'NolanArenado')

In [24]:
test_df = batting_df_norm_year[batting_df_norm_year['player_id'].isin(ids_to_test)]

In [25]:
X_test_ts = np.zeros((len(ids_to_test), 25, 13))
X_test_agg = np.zeros((len(ids_to_test), 14))

In [26]:
batting_df_norm_year.drop(columns='year').groupby('player_id').sum().sum(axis=1).sort_values()[-20:]

player_id
RyanBraun           147.967806
CurtisGranderson    148.202915
IanKinsler          149.101352
JoseAltuve          150.314690
RobinsonCano        150.588663
EdwinEncarnacion    151.978965
AnthonyRizzo        152.923240
EvanLongoria        155.213169
Shin-SooChoo        159.012314
NelsonCruz          159.476430
AdamJones           160.229929
IanDesmond          160.861983
FreddieFreeman      165.119091
PaulGoldschmidt     165.879456
BrettGardner        168.834342
JoeyVotto           169.614125
JustinUpton         172.672100
ElvisAndrus         180.491677
AndrewMcCutchen     193.335532
MikeTrout           205.874207
dtype: float64

In [27]:
for i, pid in enumerate(ids_to_test):
    p_df = test_df[test_df['player_id'] == pid].sort_values('year').drop(columns=['player_id'])
    p_df['years_played'] = p_df['year'] - p_df['year'].min() + 1
#     display(p_df)
    X_test_ts[i, :p_df['years_played'].max()] = p_df.drop(columns=['years_played', 'year']).to_numpy()
    p_df_agg = p_df.drop(columns=['year', 'years_played']).sum(axis=0)
#     display(p_df_agg)
    X_test_agg[i] = np.concatenate((p_df_agg.to_numpy(), [p_df['years_played'].max()]))

# evaluate using ensemble

In [28]:
from keras.models import load_model

In [29]:
lstm = load_model('good_lstm/')

In [30]:
list(lstm.predict(X_test_ts)[:, 1])

[0.9488754,
 0.95596063,
 0.00019762415,
 0.8618952,
 0.8649916,
 0.7616439,
 0.9311944,
 0.024338102]